In [3]:
import os
import json
from PIL import Image, ImageDraw, ImageFont
import moderngl
import numpy as np
import pandas as pd
from io import BytesIO
import math
import imageio

In [4]:
def render_pbr_material(basecolor, normal, roughness, metallic, height=None, resolution=512):
    """Render a PBR material using all texture maps"""
    
    # Create moderngl context (headless)
    ctx = moderngl.create_standalone_context(backend='egl')
    
    # Vertex shader - simple quad
    vertex_shader = '''
    #version 330
    in vec2 in_vert;
    in vec2 in_texcoord;
    out vec2 v_texcoord;
    
    void main() {
        v_texcoord = in_texcoord;
        gl_Position = vec4(in_vert, 0.0, 1.0);
    }
    '''
    
    # Fragment shader - PBR lighting
    fragment_shader = '''
    #version 330
    uniform sampler2D basecolor_tex;
    uniform sampler2D normal_tex;
    uniform sampler2D roughness_tex;
    uniform sampler2D metallic_tex;
    
    in vec2 v_texcoord;
    out vec4 fragColor;
    
    const float PI = 3.14159265359;
    
    // Simple PBR lighting model
    vec3 fresnelSchlick(float cosTheta, vec3 F0) {
        return F0 + (1.0 - F0) * pow(1.0 - cosTheta, 5.0);
    }
    
    float DistributionGGX(vec3 N, vec3 H, float roughness) {
        float a = roughness * roughness;
        float a2 = a * a;
        float NdotH = max(dot(N, H), 0.0);
        float NdotH2 = NdotH * NdotH;
        float num = a2;
        float denom = (NdotH2 * (a2 - 1.0) + 1.0);
        denom = PI * denom * denom;
        return num / denom;
    }
    
    float GeometrySchlickGGX(float NdotV, float roughness) {
        float r = (roughness + 1.0);
        float k = (r * r) / 8.0;
        float num = NdotV;
        float denom = NdotV * (1.0 - k) + k;
        return num / denom;
    }
    
    float GeometrySmith(vec3 N, vec3 V, vec3 L, float roughness) {
        float NdotV = max(dot(N, V), 0.0);
        float NdotL = max(dot(N, L), 0.0);
        float ggx2 = GeometrySchlickGGX(NdotV, roughness);
        float ggx1 = GeometrySchlickGGX(NdotL, roughness);
        return ggx1 * ggx2;
    }
    
    void main() {
        // Sample textures
        vec3 albedo = texture(basecolor_tex, v_texcoord).rgb;
        vec3 normal = texture(normal_tex, v_texcoord).rgb * 2.0 - 1.0;
        float roughness = texture(roughness_tex, v_texcoord).r;
        float metallic = texture(metallic_tex, v_texcoord).r;
        
        // Normalize normal
        normal = normalize(normal);
        
        // View and light directions
        vec3 V = normalize(vec3(0.0, 0.0, 1.0));
        // Light: 10° above the plane (grazing), azimuth 45°
        float light_elev = radians(10.0);
        vec3 L = normalize(vec3(cos(light_elev) * 0.70710678,
                                cos(light_elev) * 0.70710678,
                                sin(light_elev)));
        vec3 H = normalize(V + L);
        
        // Calculate reflectance at normal incidence
        vec3 F0 = vec3(0.04);
        F0 = mix(F0, albedo, metallic);
        
        // Cook-Torrance BRDF
        float NDF = DistributionGGX(normal, H, roughness);
        float G = GeometrySmith(normal, V, L, roughness);
        vec3 F = fresnelSchlick(max(dot(H, V), 0.0), F0);
        
        vec3 kS = F;
        vec3 kD = vec3(1.0) - kS;
        kD *= 1.0 - metallic;
        
        float NdotL = max(dot(normal, L), 0.0);
        vec3 numerator = NDF * G * F;
        float denominator = 4.0 * max(dot(normal, V), 0.0) * NdotL + 0.001;
        vec3 specular = numerator / denominator;
        
        // Light color and ambient
        vec3 lightColor = vec3(1.0);
        vec3 ambient = vec3(0.03) * albedo;
        
        vec3 Lo = (kD * albedo / PI + specular) * lightColor * NdotL;
        vec3 color = ambient + Lo;
        
        // HDR tonemapping
        color = color / (color + vec3(1.0));
        // Gamma correction
        color = pow(color, vec3(1.0/2.2));
        
        fragColor = vec4(color, 1.0);
    }
    '''
    
    # Create program
    prog = ctx.program(vertex_shader=vertex_shader, fragment_shader=fragment_shader)
    
    # Create quad vertices
    vertices = np.array([
        -1.0, -1.0,  0.0, 0.0,
         1.0, -1.0,  1.0, 0.0,
        -1.0,  1.0,  0.0, 1.0,
         1.0,  1.0,  1.0, 1.0,
    ], dtype='f4')
    
    vbo = ctx.buffer(vertices)
    vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_texcoord')
    
    # Helper to convert PIL image to texture
    def pil_to_texture(img):
        img = img.convert('RGB').resize((resolution, resolution))
        img_data = np.array(img, dtype='uint8')
        texture = ctx.texture((resolution, resolution), 3, img_data.tobytes())
        texture.filter = (moderngl.LINEAR, moderngl.LINEAR)
        return texture
    
    # Create textures
    basecolor_tex = pil_to_texture(basecolor)
    normal_tex = pil_to_texture(normal)
    roughness_tex = pil_to_texture(roughness)
    metallic_tex = pil_to_texture(metallic)
    
    # Bind textures
    basecolor_tex.use(0)
    normal_tex.use(1)
    roughness_tex.use(2)
    metallic_tex.use(3)
    
    prog['basecolor_tex'].value = 0
    prog['normal_tex'].value = 1
    prog['roughness_tex'].value = 2
    prog['metallic_tex'].value = 3
    
    # Create framebuffer
    fbo = ctx.framebuffer(
        color_attachments=[ctx.texture((resolution, resolution), 3)]
    )
    
    # Render
    fbo.use()
    ctx.clear(0.0, 0.0, 0.0)
    vao.render(moderngl.TRIANGLE_STRIP)
    
    # Read pixels
    data = fbo.read(components=3)
    img = PILImage.frombytes('RGB', (resolution, resolution), data)
    img = img.transpose(PILImage.FLIP_TOP_BOTTOM)
    
    # Cleanup
    ctx.release()
    
    return img


In [5]:
def render_pbr_turntable_camera(basecolor, normal, roughness, metallic, height=None,
                                resolution=512, frames=60, output_gif="/tmp/pbr_turntable.gif",
                                elevation_deg=45, radius=3.0, displacement_scale=0.1,
                                frame_duration=0.5):
    """
    Render a PBR material on a fixed plane while the camera orbits on a sphere
    at a constant elevation angle.
    """
    # Create headless ModernGL context
    ctx = moderngl.create_standalone_context(backend='egl')
    ctx.viewport = (0, 0, resolution, resolution)

    # Vertex shader with MVP
    vertex_shader = '''
    #version 330
    in vec3 in_vert;
    in vec2 in_texcoord;
    out vec2 v_texcoord;
    out vec3 v_world_pos;
    uniform sampler2D height_tex;
    uniform float displacement_scale;
    uniform mat4 mvp;
    void main() {
        float h = texture(height_tex, in_texcoord).r;
        vec3 pos = in_vert + vec3(0.0, 0.0, h * displacement_scale);
        v_texcoord = in_texcoord;
        v_world_pos = pos;
        gl_Position = mvp * vec4(pos, 1.0);
    }
    '''

    # Fragment shader (Cook-Torrance PBR)
    fragment_shader = '''
    #version 330
    uniform sampler2D basecolor_tex;
    uniform sampler2D normal_tex;
    uniform sampler2D roughness_tex;
    uniform sampler2D metallic_tex;
    uniform vec3 camera_pos;
    in vec2 v_texcoord;
    in vec3 v_world_pos;
    out vec4 fragColor;
    const float PI = 3.14159265359;
    vec3 fresnelSchlick(float cosTheta, vec3 F0) { return F0 + (1.0 - F0) * pow(1.0 - cosTheta, 5.0); }
    float DistributionGGX(vec3 N, vec3 H, float roughness) {
        float a = roughness * roughness; float a2 = a * a; float NdotH = max(dot(N, H), 0.0);
        float NdotH2 = NdotH * NdotH; float denom = (NdotH2 * (a2 - 1.0) + 1.0); denom = PI * denom * denom; return a2 / denom;
    }
    float GeometrySchlickGGX(float NdotV, float roughness){ float r = roughness + 1.0; float k = (r * r) / 8.0; return NdotV / (NdotV * (1.0 - k) + k); }
    float GeometrySmith(vec3 N, vec3 V, vec3 L, float roughness){ return GeometrySchlickGGX(max(dot(N, L), 0.0), roughness) * GeometrySchlickGGX(max(dot(N, V), 0.0), roughness); }
    void main(){
        vec3 albedo = texture(basecolor_tex, v_texcoord).rgb;
        vec3 n = texture(normal_tex, v_texcoord).rgb * 2.0 - 1.0;
        float rough = texture(roughness_tex, v_texcoord).r;
        float metal = texture(metallic_tex, v_texcoord).r;
        vec3 N = normalize(n);
        // View direction from camera to fragment (world space)
        vec3 V = normalize(camera_pos - v_world_pos);
        // Light: 10° above the plane (grazing), azimuth 45°
        float light_elev = radians(45.0);
        vec3 L = normalize(vec3(cos(light_elev) * 0.70710678,
                                cos(light_elev) * 0.70710678,
                                sin(light_elev)));
        vec3 H = normalize(V + L);
        vec3 F0 = vec3(0.04); F0 = mix(F0, albedo, metal);
        float NDF = DistributionGGX(N, H, rough);
        float G = GeometrySmith(N, V, L, rough);
        vec3 F = fresnelSchlick(max(dot(H, V), 0.0), F0);
        vec3 kS = F; vec3 kD = vec3(1.0) - kS; kD *= 1.0 - metal;
        float NdotL = max(dot(N, L), 0.0);
        vec3 specular = (NDF * G * F) / (4.0 * max(dot(N, V), 0.0) * NdotL + 0.001);
        vec3 Lo = (kD * albedo / PI + specular) * NdotL;
        vec3 ambient = vec3(0.03) * albedo;
        vec3 color = ambient + Lo;
        color = color / (color + vec3(1.0));
        color = pow(color, vec3(1.0 / 2.2));
        fragColor = vec4(color, 1.0);
    }
    '''

    prog = ctx.program(vertex_shader=vertex_shader, fragment_shader=fragment_shader)

    # Quad vertices (XY plane) with UV
    verts = np.array([
        [-1.0, -1.0, 0.0, 0.0, 0.0],
        [ 1.0, -1.0, 0.0, 1.0, 0.0],
        [-1.0,  1.0, 0.0, 0.0, 1.0],
        [ 1.0,  1.0, 0.0, 1.0, 1.0],
    ], dtype='f4')

    vbo = ctx.buffer(verts.tobytes())
    vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_texcoord')

    # Helper: PIL image to OpenGL texture
    def pil_to_texture(img, components=3):
        if components == 1:
            img = img.convert('L').resize((resolution, resolution))
            data = np.array(img, dtype='u1')
            tex = ctx.texture((resolution, resolution), 1, data.tobytes())
        else:
            img = img.convert('RGB').resize((resolution, resolution))
            data = np.array(img, dtype='u1')
            tex = ctx.texture((resolution, resolution), 3, data.tobytes())
        tex.filter = (moderngl.LINEAR, moderngl.LINEAR)
        return tex

    basecolor_tex = pil_to_texture(basecolor, components=3)
    normal_tex = pil_to_texture(normal, components=3)
    roughness_tex = pil_to_texture(roughness, components=3)
    metallic_tex = pil_to_texture(metallic, components=3)
    height_img = height if height is not None else Image.new('L', (resolution, resolution), 0)
    height_tex = pil_to_texture(height_img, components=1)

    basecolor_tex.use(0); normal_tex.use(1); roughness_tex.use(2); metallic_tex.use(3); height_tex.use(4)
    prog['basecolor_tex'].value = 0; prog['normal_tex'].value = 1; prog['roughness_tex'].value = 2
    prog['metallic_tex'].value = 3; prog['height_tex'].value = 4; prog['displacement_scale'].value = displacement_scale

    # Framebuffer (with depth)
    depth = ctx.depth_renderbuffer((resolution, resolution))
    fbo = ctx.framebuffer(color_attachments=[ctx.texture((resolution, resolution), 3)], depth_attachment=depth)

    # Camera helpers (row-major in NumPy; transpose on upload)
    def _normalize(v):
        n = np.linalg.norm(v)
        return v if n == 0 else v / n

    def look_at(eye, target, up):
        f = _normalize(target - eye)
        if abs(np.dot(f, up)) > 0.99:
            up = np.array([1.0, 0.0, 0.0], dtype='f4')
        s = _normalize(np.cross(f, up))
        u = np.cross(s, f)
        view = np.eye(4, dtype='f4')
        view[0, :3] = s
        view[1, :3] = u
        view[2, :3] = -f
        view[:3, 3] = -view[:3, :3] @ eye
        return view

    def perspective(fov_y_deg, aspect, near, far):
        f = 1.0 / math.tan(math.radians(fov_y_deg) * 0.5)
        proj = np.zeros((4, 4), dtype='f4')
        proj[0, 0] = f / aspect
        proj[1, 1] = f
        proj[2, 2] = (far + near) / (near - far)
        proj[2, 3] = (2 * far * near) / (near - far)
        proj[3, 2] = -1.0
        return proj

    # Text overlay setup
    try:
        font = ImageFont.truetype("DejaVuSans.ttf", 24)
    except OSError:
        font = ImageFont.load_default()
    text_color = (255, 255, 255)
    text_bg = (0, 0, 0, 160)
    padding = 4

    # Render frames
    images = []
    elev_rad = math.radians(elevation_deg)
    target = np.array([0.0, 0.0, 0.0], dtype='f4')
    world_up = np.array([0.0, 0.0, 1.0], dtype='f4')

    for i in range(frames):
        theta = 2 * math.pi * i / frames
        cam_x = radius * math.cos(theta) * math.cos(elev_rad)
        cam_y = radius * math.sin(theta) * math.cos(elev_rad)
        cam_z = radius * math.sin(elev_rad)
        eye = np.array([cam_x, cam_y, cam_z], dtype='f4')
        prog['camera_pos'].value = (float(cam_x), float(cam_y), float(cam_z))
        view = look_at(eye, target, world_up)
        proj = perspective(45.0, 1.0, 0.1, 100.0)
        mvp = proj @ view
        prog['mvp'].write(mvp.T.tobytes())

        fbo.use()
        ctx.enable(moderngl.DEPTH_TEST)
        ctx.clear(0.0, 0.0, 0.0, depth=1.0)
        vao.render(moderngl.TRIANGLE_STRIP)
        data = fbo.read(components=3)
        img = Image.frombytes('RGB', (resolution, resolution), data)
        img = img.transpose(Image.FLIP_TOP_BOTTOM)

        # Overlay angle text in top-left
        angle_deg = (360.0 * i / frames)
        text = f"Angle: {angle_deg:.1f}°"
        draw = ImageDraw.Draw(img, 'RGBA')
        text_w, text_h = draw.textbbox((0, 0), text, font=font)[2:]
        draw.rectangle([0, 0, text_w + padding * 2, text_h + padding * 2], fill=text_bg)
        draw.text((padding, padding), text, fill=text_color, font=font)

        # Repeat frames to slow playback in GIF viewers
        images.append(img)
        # images.extend([img.copy()] * max(1, int(hold_frames)))

    # Save GIF with loop and duration
    imageio.mimsave(output_gif, images, duration=[frame_duration] * len(images), loop=0)
    ctx.release()
    print("GIF saved to:", output_gif)


In [ ]:
def render_pbr_turntable_light(basecolor, normal, roughness, metallic, height=None,
                               resolution=512, frames=60, output_gif="/tmp/pbr_light_turntable.gif",
                               camera_elevation_deg=45, camera_azimuth_deg=0, camera_radius=3.0,
                               light_elevation_deg=45, light_radius=3.0, displacement_scale=0.1,
                               frame_duration=0.5):
    """
    Render a PBR material on a fixed plane while the camera is fixed at a given angle
    and a point light orbits on a sphere at a constant elevation angle.
    """
    # Create headless ModernGL context
    ctx = moderngl.create_standalone_context(backend='egl')
    ctx.viewport = (0, 0, resolution, resolution)

    # Vertex shader with MVP
    vertex_shader = '''
    #version 330
    in vec3 in_vert;
    in vec2 in_texcoord;
    out vec2 v_texcoord;
    out vec3 v_world_pos;
    uniform sampler2D height_tex;
    uniform float displacement_scale;
    uniform mat4 mvp;
    void main() {
        float h = texture(height_tex, in_texcoord).r;
        vec3 pos = in_vert + vec3(0.0, 0.0, h * displacement_scale);
        v_texcoord = in_texcoord;
        v_world_pos = pos;
        gl_Position = mvp * vec4(pos, 1.0);
    }
    '''

    # Fragment shader (Cook-Torrance PBR)
    fragment_shader = '''
    #version 330
    uniform sampler2D basecolor_tex;
    uniform sampler2D normal_tex;
    uniform sampler2D roughness_tex;
    uniform sampler2D metallic_tex;
    uniform vec3 camera_pos;
    uniform vec3 light_pos;
    in vec2 v_texcoord;
    in vec3 v_world_pos;
    out vec4 fragColor;
    const float PI = 3.14159265359;
    vec3 fresnelSchlick(float cosTheta, vec3 F0) { return F0 + (1.0 - F0) * pow(1.0 - cosTheta, 5.0); }
    float DistributionGGX(vec3 N, vec3 H, float roughness) {
        float a = roughness * roughness; float a2 = a * a; float NdotH = max(dot(N, H), 0.0);
        float NdotH2 = NdotH * NdotH; float denom = (NdotH2 * (a2 - 1.0) + 1.0); denom = PI * denom * denom; return a2 / denom;
    }
    float GeometrySchlickGGX(float NdotV, float roughness){ float r = roughness + 1.0; float k = (r * r) / 8.0; return NdotV / (NdotV * (1.0 - k) + k); }
    float GeometrySmith(vec3 N, vec3 V, vec3 L, float roughness){ return GeometrySchlickGGX(max(dot(N, L), 0.0), roughness) * GeometrySchlickGGX(max(dot(N, V), 0.0), roughness); }
    void main(){
        vec3 albedo = texture(basecolor_tex, v_texcoord).rgb;
        vec3 n = texture(normal_tex, v_texcoord).rgb * 2.0 - 1.0;
        float rough = texture(roughness_tex, v_texcoord).r;
        float metal = texture(metallic_tex, v_texcoord).r;
        vec3 N = normalize(n);
        // View direction from camera to fragment (world space)
        vec3 V = normalize(camera_pos - v_world_pos);
        // Point light on sphere (world space)
        vec3 L = normalize(light_pos - v_world_pos);
        vec3 H = normalize(V + L);
        vec3 F0 = vec3(0.04); F0 = mix(F0, albedo, metal);
        float NDF = DistributionGGX(N, H, rough);
        float G = GeometrySmith(N, V, L, rough);
        vec3 F = fresnelSchlick(max(dot(H, V), 0.0), F0);
        vec3 kS = F; vec3 kD = vec3(1.0) - kS; kD *= 1.0 - metal;
        float NdotL = max(dot(N, L), 0.0);
        vec3 specular = (NDF * G * F) / (4.0 * max(dot(N, V), 0.0) * NdotL + 0.001);
        vec3 Lo = (kD * albedo / PI + specular) * NdotL;
        vec3 ambient = vec3(0.03) * albedo;
        vec3 color = ambient + Lo;
        color = color / (color + vec3(1.0));
        color = pow(color, vec3(1.0 / 2.2));
        fragColor = vec4(color, 1.0);
    }
    '''

    prog = ctx.program(vertex_shader=vertex_shader, fragment_shader=fragment_shader)

    # Quad vertices (XY plane) with UV
    verts = np.array([
        [-1.0, -1.0, 0.0, 0.0, 1.0],
        [ 1.0, -1.0, 0.0, 0.0, 0.0],
        [-1.0,  1.0, 0.0, 1.0, 1.0],
        [ 1.0,  1.0, 0.0, 1.0, 0.0],
    ], dtype='f4')

    vbo = ctx.buffer(verts.tobytes())
    vao = ctx.simple_vertex_array(prog, vbo, 'in_vert', 'in_texcoord')

    # Helper: PIL image to OpenGL texture
    def pil_to_texture(img, components=3):
        if components == 1:
            img = img.convert('L').resize((resolution, resolution))
            data = np.array(img, dtype='u1')
            tex = ctx.texture((resolution, resolution), 1, data.tobytes())
        else:
            img = img.convert('RGB').resize((resolution, resolution))
            data = np.array(img, dtype='u1')
            tex = ctx.texture((resolution, resolution), 3, data.tobytes())
        tex.filter = (moderngl.LINEAR, moderngl.LINEAR)
        return tex

    basecolor_tex = pil_to_texture(basecolor, components=3)
    normal_tex = pil_to_texture(normal, components=3)
    roughness_tex = pil_to_texture(roughness, components=3)
    metallic_tex = pil_to_texture(metallic, components=3)
    height_img = height if height is not None else Image.new('L', (resolution, resolution), 0)
    height_tex = pil_to_texture(height_img, components=1)

    basecolor_tex.use(0); normal_tex.use(1); roughness_tex.use(2); metallic_tex.use(3); height_tex.use(4)
    prog['basecolor_tex'].value = 0; prog['normal_tex'].value = 1; prog['roughness_tex'].value = 2
    prog['metallic_tex'].value = 3; prog['height_tex'].value = 4; prog['displacement_scale'].value = displacement_scale

    # Framebuffer (with depth)
    depth = ctx.depth_renderbuffer((resolution, resolution))
    fbo = ctx.framebuffer(color_attachments=[ctx.texture((resolution, resolution), 3)], depth_attachment=depth)

    # Camera helpers (row-major in NumPy; transpose on upload)
    def _normalize(v):
        n = np.linalg.norm(v)
        return v if n == 0 else v / n

    def look_at(eye, target, up):
        f = _normalize(target - eye)
        if abs(np.dot(f, up)) > 0.99:
            up = np.array([1.0, 0.0, 0.0], dtype='f4')
        s = _normalize(np.cross(f, up))
        u = np.cross(s, f)
        view = np.eye(4, dtype='f4')
        view[0, :3] = s
        view[1, :3] = u
        view[2, :3] = -f
        view[:3, 3] = -view[:3, :3] @ eye
        return view

    def perspective(fov_y_deg, aspect, near, far):
        f = 1.0 / math.tan(math.radians(fov_y_deg) * 0.5)
        proj = np.zeros((4, 4), dtype='f4')
        proj[0, 0] = f / aspect
        proj[1, 1] = f
        proj[2, 2] = (far + near) / (near - far)
        proj[2, 3] = (2 * far * near) / (near - far)
        proj[3, 2] = -1.0
        return proj

    # Text overlay setup
    # try:
    #     font = ImageFont.truetype("DejaVuSans.ttf", 24)
    # except OSError:
    #     font = ImageFont.load_default()
    # text_color = (255, 255, 255)
    # text_bg = (0, 0, 0, 160)
    # padding = 4

    # Fixed camera pose
    cam_elev = math.radians(camera_elevation_deg)
    cam_az = math.radians(camera_azimuth_deg)
    cam_x = camera_radius * math.cos(cam_az) * math.cos(cam_elev)
    cam_y = camera_radius * math.sin(cam_az) * math.cos(cam_elev)
    cam_z = camera_radius * math.sin(cam_elev)
    camera_pos = np.array([cam_x, cam_y, cam_z], dtype='f4')
    prog['camera_pos'].value = (float(cam_x), float(cam_y), float(cam_z))

    target = np.array([0.0, 0.0, 0.0], dtype='f4')
    world_up = np.array([0.0, 0.0, 1.0], dtype='f4')
    view = look_at(camera_pos, target, world_up)
    proj = perspective(45.0, 1.0, 0.1, 100.0)
    mvp = proj @ view
    prog['mvp'].write(mvp.T.tobytes())

    # Render frames
    images = []
    light_elev = math.radians(light_elevation_deg)

    for i in range(frames):
        theta = 2 * math.pi * i / frames
        light_x = light_radius * math.cos(theta) * math.cos(light_elev)
        light_y = light_radius * math.sin(theta) * math.cos(light_elev)
        light_z = light_radius * math.sin(light_elev)
        prog['light_pos'].value = (float(light_x), float(light_y), float(light_z))

        fbo.use()
        ctx.enable(moderngl.DEPTH_TEST)
        ctx.clear(0.0, 0.0, 0.0, depth=1.0)
        vao.render(moderngl.TRIANGLE_STRIP)
        data = fbo.read(components=3)
        img = Image.frombytes('RGB', (resolution, resolution), data)
        img = img.transpose(Image.FLIP_TOP_BOTTOM)

        # Overlay angle text in top-left
        # angle_deg = (360.0 * i / frames)
        # text = f"Light Angle: {angle_deg:.1f}°"
        # draw = ImageDraw.Draw(img, 'RGBA')
        # text_w, text_h = draw.textbbox((0, 0), text, font=font)[2:]
        # draw.rectangle([0, 0, text_w + padding * 2, text_h + padding * 2], fill=text_bg)
        # draw.text((padding, padding), text, fill=text_color, font=font)

        images.append(img)

    # Save GIF with loop and duration
    imageio.mimsave(output_gif, images, duration=[frame_duration] * len(images), loop=0)
    ctx.release()
    print("GIF saved to:", output_gif)

In [161]:
matsynth_df = pd.read_parquet('/home/woody/vlgm/vlgm116v/MatSynth/data/test-00001-of-00008.parquet')
# display(matsynth_df.head())

In [1]:
# idx = 4
# basecolor = PILImage.open(BytesIO(matsynth_df['basecolor'][idx]['bytes'])).resize((1024, 1024))
# normal = PILImage.open(BytesIO(matsynth_df['normal'][idx]['bytes'])).resize((1024, 1024))
# roughness = PILImage.open(BytesIO(matsynth_df['roughness'][idx]['bytes'])).resize((1024, 1024))
# metallic = PILImage.open(BytesIO(matsynth_df['metallic'][idx]['bytes'])).resize((1024, 1024))
# height = PILImage.open(BytesIO(matsynth_df['height'][idx]['bytes'])).resize((1024, 1024))
# 
# rendered_img = render_pbr_material(basecolor, normal, roughness, metallic, height)
# display(rendered_img)

In [123]:
name = "circuit"
basecolor = Image.open(f'output/{name}/albedo.png')
normal = Image.open(f'output/{name}/normal.png')
roughness = Image.open(f'output/{name}/roughness.png')
metallic = Image.open(f'output/{name}/metallic.png')
height = Image.open(f'output/{name}/height.png')

In [ ]:
# render_pbr_turntable(basecolor, normal, roughness, metallic, height, resolution=1024, frames=4, output_gif="temp.gif")

GIF saved to: temp.gif


In [124]:
render_pbr_turntable_camera(basecolor, normal, roughness, metallic, height=height,
                             resolution=1024, frames=30, output_gif="temp.gif",
                             elevation_deg=45, radius=3.0, frame_duration=10)

GIF saved to: temp.gif


In [125]:
render_pbr_turntable_light(basecolor, normal, roughness, metallic, height=height,
                             resolution=1024, frames=30, output_gif="temp.gif",
                             camera_elevation_deg=90, camera_azimuth_deg=0, camera_radius=2.0,
                             light_elevation_deg=65, light_radius=3.0, frame_duration=10)

GIF saved to: temp.gif
